In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    log_loss,
    roc_auc_score,
    recall_score,
    precision_score,
    accuracy_score,
    plot_roc_curve,
    plot_confusion_matrix,
    roc_curve,
    confusion_matrix,
)
import itertools
from tensorflow.keras.initializers import Constant, TruncatedNormal
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from numpy import mean, absolute

Learning Bias Mitigation: HLE <> DL

In [ ]:
## define count of n from temporal models
n = list(range(2, 61))

Models

In [ ]:
## define metrics dataframe
metrics = pd.DataFrame(
    columns=[
        "model",
        "group",
        "subgroup",
        "Length",
        "Sentence",
        "Accuracy",
        "loss",
        "optimizer",
        "metrics",
        "Precision",
        "Recall",
        "AUC",
        "FPR",
    ]
)

In [ ]:
## define feature cols
feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps",
]

In [ ]:
optimizer = [
    "Adam",
    "Adam",
    "Adam",
    "SGD",
    "SGD",
    "SGD",
    "Adam",
    "Adam",
    "Adam",
    "SGD",
    "SGD",
    "SGD",
]
loss = [
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
]
metrics_ = [
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "AUC",
    "AUC",
    "AUC",
    "AUC",
    "AUC",
    "AUC",
]

""""
build dropout prediction model
"""


def build_model():
    model = Sequential()
    model.add(Dense(24, input_dim=24, activation="relu"))
    model.add(Dense(48, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(12, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    return model


"""
calculate and extract relevant metrics from y and pred
return metrics
"""


def get_dn_metrics(model, X, y):
    yhat_probs = model.predict(X, verbose=0)
    yhat_classes = (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]
    a = accuracy_score(y, yhat_classes)
    p = precision_score(y, yhat_classes)
    r = recall_score(y, yhat_classes)
    roc_auc = roc_auc_score(y, yhat_probs)
    tn, fp, fn, tp = confusion_matrix(y, yhat_classes).ravel()
    fpr = fp / (fp + tn)

    return a, p, r, roc_auc, fpr


# loop through matrices
for loss, optimizer, metrics_ in zip(loss, optimizer, metrics_):
    for i in n:
        path = "buecher_allsessions/matrix" + str(i) + ".pkl"
        infile = open(path, "rb")
        df = pickle.load(infile)
        infile.close()
        df = df.reset_index()

        y_len = len(feature_cols)
        X = df[feature_cols].astype(float)
        y = df.y
        y = y.astype("int")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=1
        )

        model = build_model()

        model.compile(loss=loss, optimizer=optimizer, metrics=[metrics_])

        model.fit(
            x=X_train,
            y=y_train,
            epochs=10,
            batch_size=128,
            verbose=0,
            validation_data=(X_test, y_test),
        )

        scores = model.evaluate(x=X_test, y=y_test, verbose=0)

        a, p, r, roc_auc, fpr = get_dn_metrics(model, X_test, y_test)
        metrics = metrics.append(
            {
                "model": "DL",
                "group": "all",
                "subgroup": "all",
                "Length": len(df),
                "Sentence": i,
                "Accuracy": a,
                "loss": loss,
                "optimizer": optimizer,
                "metrics": metrics_,
                "Precision": p,
                "Recall": r,
                "AUC": roc_auc,
                "FPR": fpr,
            },
            ignore_index=True,
        )

        group = ["buecher", "buecher"]
        subgroup = ["buch0", "buch1"]
        matrice = ["matrices_buecher_0", "matrices_buecher_1"]

        for group, subgroup, matrix in zip(group, subgroup, matrice):
            path = (
                "../../02_dropout_prediction/01_keep_it_up/"
                + matrix
                + "/matrix"
                + str(i)
                + ".pkl"
            )
            infile = open(path, "rb")
            df = pickle.load(infile)
            infile.close()
            df = df.reset_index()
            y_len = len(feature_cols)
            X = df[feature_cols].astype(float)
            y = df.y
            y = y.astype("int")

            a, p, r, roc_auc, fpr = get_dn_metrics(model, X, y)

            metrics = metrics.append(
                {
                    "model": "DL",
                    "group": group,
                    "subgroup": subgroup,
                    "Length": len(df),
                    "Sentence": i,
                    "Accuracy": a,
                    "loss": loss,
                    "optimizer": optimizer,
                    "metrics": metrics_,
                    "Precision": p,
                    "Recall": r,
                    "AUC": roc_auc,
                    "FPR": fpr,
                },
                ignore_index=True,
            )

Evaluate

In [ ]:
## construct dfs from metric df
grouped = metrics.groupby(df.group)
df_buecher = grouped.get_group("buecher")

df_buecher = df_buecher.drop(columns=["group", "Accuracy"])
df_buecher = pd.pivot_table(
    df_buecher,
    values=["Precision", "Recall", "AUC", "FPR"],
    index=["loss", "optimizer", "metrics", "Sentence"],
    columns=["subgroup"],
)
df_buecher["PP"] = df_buecher.Precision.buch1 - df_buecher.Precision.buch0
df_buecher["EO"] = df_buecher.Recall.buch0 - df_buecher.Recall.buch1
df_buecher["SA"] = df_buecher.AUC.buch1 - df_buecher.AUC.buch0
df_buecher["PE"] = df_buecher.FPR.buch0 - df_buecher.FPR.buch1
df_buecher = df_buecher.drop(columns=["AUC", "Precision", "Recall", "FPR"])
df_buecher.columns = df_buecher.columns.droplevel(1)
df_buecher = pd.pivot_table(
    df_buecher,
    values=["PP", "EO", "SA", "PE"],
    index=["Sentence"],
    columns=["loss", "optimizer", "metrics"],
)

In [ ]:
## claculate mean of the results and map in data frame
met = ["EO", "PE", "PP", "SA"]
fertig = pd.DataFrame()

optimizer = [
    "Adam",
    "Adam",
    "Adam",
    "SGD",
    "SGD",
    "SGD",
    "Adam",
    "Adam",
    "Adam",
    "SGD",
    "SGD",
    "SGD",
]
loss = [
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
    "binary_crossentropy",
    "MeanSquaredError",
    "Hinge",
]
metrics_ = [
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "Accuracy",
    "AUC",
    "AUC",
    "AUC",
    "AUC",
    "AUC",
    "AUC",
]

for loss, optimizer, metrics_ in zip(loss, optimizer, metrics_):
    for metric in metrics:
        for i, j in [(2, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60)]:
            a = np.mean(df_buecher[metric][loss][optimizer][metrics_][i:j])
            temp = pd.DataFrame(
                {
                    "Metrik": [metric],
                    "Model": "DL",
                    "Range": f"{i:02d}-{j-1:02d}",
                    "Val": a,
                    "loss": loss,
                    "optimizer": optimizer,
                    "metrics_": metrics_,
                }
            )
            fertig = pd.concat([fertig, temp])

In [ ]:
"""
functions to format results
set two threshols: one at |0.02| in orange and one at |0.05| in red
format all negative values in bold
"""


def threshold001(v, props=""):
    return props if (v > 0.02) or (v < -0.02) else None


def threshold005(v, props=""):
    return props if (v > 0.05) or (v < -0.05) else None


def negativeValue(v, props=""):
    return props if (v < 0) else None


def showTable(df):
    styled = (
        df.style.set_properties(color="black", align="right")
        .set_properties(**{"background-color": "white"})
        .applymap(threshold001, props="color:orange;")
        .applymap(threshold005, props="color:red;")
        .applymap(negativeValue, props="font-weight:bold;")
    )
    return styled

In [ ]:
## show results table
mean_table = pd.pivot_table(
    fertig,
    values=["Val"],
    index=["loss", "optimizer", "metrics_", "Range"],
    columns=["Metrik", "Model"],
)
showTable(mean_table)

In [ ]:
metrics

In [ ]:
## code to print results of specific measures
# grouped = metrics.groupby(metrics.group)
# df_all = grouped.get_group("all")
# modell = df_all.groupby(df_all.loss)
# five = modell.get_group('MeanSquaredError')
# n = five.groupby(five.optimizer)
# n = n.get_group('SGD')
# f = n.groupby(n.metrics)
# f = f.get_group('AUC')
# ax = sns.lineplot(data=f, x='Sentence', y='Accuracy', hue='model')